# Segmenting and Clustering Neighborhoods in Toronto

project explanation:

# Goal: 
John is going to change his house in Toronto but he prefers living in the three boroughs of Central Toronto, East Toronto and Downtown Toronto which are near each other. he likes his previous neighborhood and seeks for a similar neighborhood. in this project we are going to use informations about neighborhoods in these boroughs alongside what we can get about their venues form Foursquare.com to cluster them. we wish to help John choose a neighborhood similar to where he is living now. 

# process:
0- installing and importing the required libraries

1- getting postal codes of Toronto - Canada from *Wikipedia.org* (including postal code - boroughs and neighborhoods) - making a dataframe out of it

2- downloading and reading a CSV file containing the postal codes and their latitudes and longitudes - making a dataframe out of it

3- cleaning and merging the two dataframes 

4- choosing a sample of data containing only the three boroughs (Central, East and Downtown Toronto)

5- exploring the venues in each neighborhood using the *fousqaure*. we will use these venues for clustering the neighborhoods.

6- clustering the neighborhoods based on their venues categories and frequencies using *Kmeans* clustering.

7- visualizing the clusters using *folium*


## 0- installing and importing the required libraries

In [1]:
import pandas as pd #library for data analysis
import numpy as np # library for working with vectors
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans # for clustering
import matplotlib.cm as cm  #for better visualizing the results clusters by folium
import matplotlib.colors as colors  #for better visualizing the results clusters by folium

#installing beautifulsoup for website scraping 
!pip install beautifulsoup4
!pip install lxml html5lib beautifulsoup4
from bs4 import BeautifulSoup

!conda install -c conda-forge folium=0.5.0 --yes #installing folium (if not installed already) - map rendering library 
import folium # map rendering library

!pip install urllib3
from urllib.request import urlopen # Opening a network object denoted by a URL for reading

!pip install requests # to get url
import requests # library to handle requests

!pip install geopy 
from geopy.geocoders import Nominatim # a module to the lat and lng of a given address

!pip install geocoder
print('libraries are intalled and imported')

Solving environment: done

# All requested packages already installed.

libraries are intalled and imported


## 1 - getting postal codes of Toronto - Canada from *Wikipedia.org*

In [3]:
def getHTMLContent(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In the **getHTMLContent()** function, we pass in the URL. Here, we first open the url using the **urlopen** method. This enables us to apply **BeautifulSoup** library to get the HTML using a parser.

In [4]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
content = getHTMLContent(link)
tables = content.find_all('table')

checking the content of tables, we find that our desired data is in the table -- class : 'wikitable sortable' -- 

In [5]:
table = content.find('table', {'class': 'wikitable sortable'})
#table

A table in HTML is comprised of rows denoted by the tags **tr** .  Each row has cells which can either be headings defined using <th></th> **th** or data defined using <td></td> **td**. we use them to get the data needed. 

we append the postal codes, boroughs and neighborhoods to three lists of A, B, C  respectively and then make a dataframe out of them.

In [6]:
A=[]
B=[]
C=[]

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) ==3:
        A.append(cells[0].find(text = True))
        B.append(cells[1].find(text = True))
        C.append(cells[2].find(text = True))


In [7]:
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
df

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
...,...,...,...
175,M5Z\n,Not assigned\n,Not assigned\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."


removing the "\n" form our dataframe items.

In [8]:
df.replace(regex=['\n'], value='', inplace = True)
df

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


removing the rows where the boroughs are not assigned. 

In [9]:
df2 = df[df['Borough']!= 'Not assigned']
df2

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


reseting the index

In [10]:
df2.reset_index(drop = True, inplace = True)
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [48]:
df2.describe()

,PostalCode,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M4G,North York,Downsview
freq,1,24,4


In [11]:
df2[df2['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


we checked to know if there is any "Not assigned" value in the "Neighborhood" and there was none. in case there is there is "Not assigned" value, we can use the following codes to replace the "Not assigned" values the Borough in the same row. 

In [12]:
idx = df2.index[df2['Neighborhood']=='Not assigned']
for index in idx:
    df2["Neighborhood"][index] = df["Borough"][index]

sorting the df by postal code.

In [14]:
df_sorted = df2.sort_values(by =['PostalCode'])
df_sorted.reset_index(inplace = True, drop = True)
df_sorted

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


checking the shape of the dataframe

In [15]:
df_sorted.shape

(103, 3)

### 2- doanloading and reading the 'CSV' file containing the latitude and longitude of different postal codes in Toronto

In [16]:
!wget -q -O 'toronto_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [17]:
toronto_data = pd.read_csv('toronto_data.csv')
# rename the column Postal Code to PostalCode to be like how it is written in our first dataframe
toronto_data.rename(columns ={'Postal Code':'PostalCode'}, inplace = True)
toronto_data

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [18]:
toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PostalCode  103 non-null    object 
 1   Latitude    103 non-null    float64
 2   Longitude   103 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.5+ KB


 ## 3- merging the two dataframes

In [22]:
Toronto = pd.merge(df_sorted, toronto_data, left_on='PostalCode', right_on='PostalCode')
Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [23]:
Toronto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PostalCode    103 non-null    object 
 1   Borough       103 non-null    object 
 2   Neighborhood  103 non-null    object 
 3   Latitude      103 non-null    float64
 4   Longitude     103 non-null    float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


## 4- choosing desired sample of data (east, central and downtown boroughs)

In [24]:
Toronto_smpl = Toronto[(Toronto['Borough'] == 'Downtown Toronto') |
                      (Toronto['Borough'] == 'Central Toronto')|(Toronto['Borough'] == 'East Toronto')]
Toronto_smpl

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [25]:
Toronto_smpl.shape

(33, 5)

In [26]:
Toronto_smpl.sort_values(by = ['Borough'], inplace = True)
Toronto_smpl.reset_index(drop = True, inplace = True)
Toronto_smpl

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
1,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M4S,Central Toronto,Davisville,43.704324,-79.388790
6,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
7,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
8,M5N,Central Toronto,Roselawn,43.711695,-79.416936
9,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [27]:
Toronto_smpl['Neighborhood'].describe()

count                                                    33
unique                                                   33
top       Business reply mail Processing Centre, South C...
freq                                                      1
Name: Neighborhood, dtype: object

## 5- exploring the venues in each neighborhood using the fousqaure

first we check the response for one neighborhood. Then, based on that, we provide a function to return the venues for all neighborhoods in our data.

In [28]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
to_latitude = location.latitude
to_longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(to_latitude, to_longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


In [31]:
neighborhood_latitude = Toronto_smpl.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_smpl.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Toronto_smpl.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Annex, North Midtown, Yorkville are 43.6727097, -79.40567840000001.


importing the foursqaure credebtials

In [29]:
CLIENT_ID = 'TZUWG4AZN1DFC0ETKYXJ1RSWWL43ZRCGNS4QH13KW1BGEWHQ' # your Foursquare ID
CLIENT_SECRET = 'I4MBTDMNQ3UGD22ZF4DH1GHQOPRZ1XNE1UXTRK5CA02NU535' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TZUWG4AZN1DFC0ETKYXJ1RSWWL43ZRCGNS4QH13KW1BGEWHQ
CLIENT_SECRET:I4MBTDMNQ3UGD22ZF4DH1GHQOPRZ1XNE1UXTRK5CA02NU535


setting the url for getting information about the venues from foursqaure

In [32]:
radius =500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,neighborhood_latitude,neighborhood_longitude,VERSION,radius,LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=TZUWG4AZN1DFC0ETKYXJ1RSWWL43ZRCGNS4QH13KW1BGEWHQ&client_secret=I4MBTDMNQ3UGD22ZF4DH1GHQOPRZ1XNE1UXTRK5CA02NU535&ll=43.6727097,-79.40567840000001&v=20180604&radius=500&limit=100'

In [33]:
results = requests.get(url)
result1 = results.json()
#result1

defining a function to return venues for neighborhoods

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET,lat,lng,VERSION,radius,LIMIT)

      
        results1 = requests.get(url).json()
        response1 = results1['response']
        response2 = response1
        venues1 = response2['venues']
        #return only relevant information for each nearby venue
        for i in range(len(venues1)):
            venue_i = venues1[i]
            venue_loc = venue_i['location']
            venue_cat = venue_i['categories']
            venue_name = venue_i['name']
            venue_lat = venue_loc['lat']
            venue_lng = venue_loc['lng'] 
            venues_list.insert(i, (name,lat,lng, venue_name, venue_lat,venue_lng,venue_cat))
        
    nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [35]:
Toronto_smpl_venues = getNearbyVenues(names=Toronto_smpl['Neighborhood'],
                                   latitudes=Toronto_smpl['Latitude'],
                                   longitudes=Toronto_smpl['Longitude'])

The Annex, North Midtown, Yorkville
Forest Hill North & West, Forest Hill Road Park
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Berczy Park
University of Toronto, Harbord
Commerce Court, Victoria Hotel
Toronto Dominion Centre, Design Exchange
Harbourfront East, Union Station, Toronto Islands
Richmond, Adelaide, King
Central Bay Street
Regent Park, Harbourfront
St. James Town
Garden District, Ryerson
Kensington Market, Chinatown, Grange Park
Church and Wellesley
St. James Town, Cabbagetown
Rosedale
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Queen's Park, Ontario Provincial Government
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
The Beaches
India Bazaar, The Beaches West
The Danforth West, Riverdale
Studio District
Business reply mail Processing Centre, South Central Letter

checking the results of the venues for neighborhoods

In [36]:
print(Toronto_smpl_venues.shape)
Toronto_smpl_venues.head(10)

(3167, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Canada Post South Central Plant,43.663323,-79.321005,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G..."
1,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Canada Post,43.660682,-79.327976,[]
2,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,"[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L..."
3,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Starbucks,43.659600,-79.328277,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C..."
4,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Amin Car Repair Garage,43.663544,-79.320130,"[{'id': '56aa371be4b08b9a8d5734d3', 'name': 'A..."
5,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Jonathan Ashbridge Park,43.664702,-79.319898,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P..."
6,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Ashbridges Bay Waste Water Treatment Plant,43.658700,-79.320867,"[{'id': '4eb1bea83b7b6f98df247e06', 'name': 'F..."
7,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Tubs and Gee Gage Field,43.663048,-79.319162,"[{'id': '4bf58dd8d48988d15f941735', 'name': 'F..."
8,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Access Storage - Downtown Toronto,43.661974,-79.319460,"[{'id': '4f04b1572fb6e1c99f3db0bf', 'name': 'S..."
9,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Joao Carvalho Mastering,43.662775,-79.326446,[]


the venue category includes id, name and other items. we just need the name, the following function is written for extracting the **'name'** from the Veneu Category.

In [37]:
# function that extracts the category of the venue
def get_category_type1(row):
    try:
        categories_list = row['Venue Category']
    except:
        categories_list = row['Venue Category']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [38]:
Toronto_smpl_venues['Venue Category'] = Toronto_smpl_venues.apply(get_category_type1, axis=1)
Toronto_smpl_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Canada Post South Central Plant,43.663323,-79.321005,Government Building
1,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Canada Post,43.660682,-79.327976,None
2,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,TTC Russell Division,43.664908,-79.322560,Light Rail Station
3,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Starbucks,43.659600,-79.328277,Coffee Shop
4,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Amin Car Repair Garage,43.663544,-79.320130,Auto Workshop


checking th shape

In [34]:
Toronto_smpl_venues.shape

(3167, 7)

checking how many venues were returned for each neighborhood

In [39]:
Toronto_smpl_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",92,92,92,92,92,79
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",63,63,63,63,63,62
Central Bay Street,100,100,100,100,100,95
Christie,100,100,100,100,100,94
Church and Wellesley,100,100,100,100,100,90
"Commerce Court, Victoria Hotel",100,100,100,100,100,93
Davisville,100,100,100,100,100,79
Davisville North,100,100,100,100,100,71


#### finding out how many unique categories can be curated from all the returned venues

In [41]:
print('There are {} uniques categories.'.format(len(Toronto_smpl_venues['Venue Category'].unique())))

There are 389 uniques categories.


### Analyze each neighborhood

In [42]:
# one hot encoding
Toronto_smpl_onehot = pd.get_dummies(Toronto_smpl_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_smpl_onehot['Neighborhood'] = Toronto_smpl_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_smpl_onehot.columns[-1]] + list(Toronto_smpl_onehot.columns[:-1])
Toronto_smpl_onehot = Toronto_smpl_onehot[fixed_columns]

Toronto_smpl_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Vietnamese Restaurant,Voting Booth,Watch Shop,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
Toronto_smpl_onehot.columns

Index(['Neighborhood', 'ATM', 'Accessories Store', 'Adult Boutique',
       'Advertising Agency', 'Afghan Restaurant', 'Airport',
       'Airport Food Court', 'Airport Gate', 'Airport Lounge',
       ...
       'Vietnamese Restaurant', 'Voting Booth', 'Watch Shop', 'Wine Bar',
       'Wine Shop', 'Winery', 'Wings Joint', 'Women's Store', 'Yoga Studio',
       'Zoo Exhibit'],
      dtype='object', length=389)

as we can not see all columns, we change the maximum number of columns to be displayed to 400.

In [44]:
pd.set_option('max_columns',400)
Toronto_smpl_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Camera Store,Campaign Office,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Theater,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Corporate Cafeteria,Corporate Coffee Shop,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Driving School,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Monument / Landmark,Moroccan Restaurant,Mosque,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Nursery School,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculp

### grouping by neighborhood and taking the mean of the frequency of occurrence of each category

In [47]:
Toronto_smpl_grouped = Toronto_smpl_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_smpl_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Baggage Claim,Bakery,Ballroom,Bank,Bar,Baseball Field,Basketball Court,Beach,Bed & Breakfast,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Camera Store,Campaign Office,Candy Store,Capitol Building,Car Wash,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,City Hall,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Administrative Building,College Arts Building,College Auditorium,College Bookstore,College Cafeteria,College Classroom,College Communications Building,College Engineering Building,College Gym,College History Building,College Lab,College Library,College Quad,College Rec Center,College Residence Hall,College Science Building,College Soccer Field,College Theater,College Track,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Community College,Concert Hall,Conference Room,Convenience Store,Convention Center,Corporate Cafeteria,Corporate Coffee Shop,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cupcake Shop,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Driving School,Dry Cleaner,Dumpling Restaurant,EV Charging Station,Eastern European Restaurant,Electronics Store,Elementary School,Embassy / Consulate,Emergency Room,Entertainment Service,Ethiopian Restaurant,Event Space,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Fire Station,Fish & Chips Shop,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Frame Store,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Herbs & Spices Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hospital Ward,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Insurance Office,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Laundry Service,Lawyer,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Medical Center,Medical Lab,Medical School,Mediterranean Restaurant,Meeting Room,Men's Store,Mental Health Office,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Middle School,Military Base,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Mobility Store,Monument / Landmark,Moroccan Restaurant,Mosque,Movie Theater,Moving Target,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Nightlife Spot,Non-Profit,Nursery School,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculp

In [48]:
Toronto_smpl_grouped.shape

(33, 389)

a function to sort revenues in descending order

In [49]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

returning the **top ten** most commmon venues for each neighborhood

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_smpl_grouped['Neighborhood']

for ind in np.arange(Toronto_smpl_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_smpl_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Office,Building,Residential Building (Apartment / Condo),Tech Startup,Parking,Laundry Service,Hotel,Breakfast Spot,Food Truck,Movie Theater
1,"Business reply mail Processing Centre, South C...",Building,Office,Light Rail Station,Gym / Fitness Center,Athletics & Sports,Theater,Butcher,Restaurant,Rental Car Location,Antique Shop
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Gate,Airport Service,Moving Target,Airport Terminal,General Travel,Coffee Shop,Airport Lounge,Boat or Ferry,Harbor / Marina,Park
3,Central Bay Street,Hospital,Hospital Ward,Coffee Shop,Medical Center,Office,Emergency Room,Pharmacy,Food Court,Japanese Restaurant,Café
4,Christie,Office,Café,Design Studio,Laundry Service,Bakery,Furniture / Home Store,Grocery Store,Gym / Fitness Center,Building,Karaoke Bar


# 6- clustering neighborhoods

Now, we are going to use **Kmeans** to cluster the neighborhoods in a sample of Toronto Boroughs into **5** clusters.

In [51]:
# set number of clusters
kclusters = 5

Toronto_smpl_grouped_clustering = Toronto_smpl_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_smpl_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 4, 2, 0, 3, 0, 1, 3, 1], dtype=int32)

creating a new dataframe that includes the **cluster** as well as the **top 10 venues** for each neighborhood.

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_smpl_merged = Toronto_smpl

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_smpl_merged = Toronto_smpl_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_smpl_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,3,Residential Building (Apartment / Condo),Office,Bed & Breakfast,Building,General Entertainment,Gym,Playground,Japanese Restaurant,Speakeasy,Metro Station
1,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Office,Residential Building (Apartment / Condo),Doctor's Office,General Entertainment,Tech Startup,Park,High School,Student Center,Jewelry Store,Miscellaneous Shop
2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,College Classroom,School,Bus Line,Housing Development,General Entertainment,Office,Bank,Building,Salon / Barbershop,Pool
3,M4P,Central Toronto,Davisville North,43.712751,-79.390197,3,Residential Building (Apartment / Condo),Office,Dog Run,Playground,Building,Medical Center,Scenic Lookout,Strip Club,Hotel,Breakfast Spot
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,1,Shoe Store,Health & Beauty Service,Cosmetics Shop,Clothing Store,Sporting Goods Shop,Boutique,Kids Store,Dentist's Office,Men's Store,Metro Station


# 7- visualizing the resulting clusters

## **A picture is worth a thousand words**

#### now, it is time to visualize the resulting clusters

In [53]:
# create map
map_clusters = folium.Map(location=[to_latitude, to_longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1.2, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, nbr, brh, cluster in zip(Toronto_smpl_merged['Latitude'], Toronto_smpl_merged['Longitude'], Toronto_smpl_merged['Neighborhood'],
                                       Toronto_smpl_merged['Borough'], Toronto_smpl_merged['Cluster Labels']):
    label = folium.Popup(str(nbr) +' '+ str(brh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters